In [ ]:
!pip install faiss-cpu sentence_transformers

In [ ]:
! pip install langgraph langchain-community pypdf faiss-cpu sentence-transformers langchain-groq

In [ ]:
!pip install langchain_huggingface

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader


In [7]:
loader = PyPDFLoader("/content/transformer_without_normalization.pdf")
pages = loader.load_and_split()

In [8]:
# documents = TextLoader("../../how_to/state_of_the_union.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(pages)
embeddingsModel = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-dot-v5"
)
retriever = FAISS.from_documents(texts, embeddingsModel).as_retriever(
    search_kwargs={"k": 20}
)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.27k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
query = "What is Normalization?"
docs = retriever.invoke(query)

In [10]:
print(docs)

[Document(id='e20025fd-8477-4945-9d26-cf4f1fce1ca1', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-03-14T01:17:11+00:00', 'author': '', 'keywords': '', 'moddate': '2025-03-14T01:17:11+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/transformer_without_normalization.pdf', 'total_pages': 19, 'page': 1, 'page_label': '2'}, page_content='normalization (Ulyanov et al., 2016), were initially proposed for specialized tasks such as object detection and\nimage stylization. They share the same overall formulation but differ in the axes and ranges over which the\nstatistics are computed.\nLayer normalization (LN) (Ba et al., 2016) and root mean square normalization (RMSNorm) (Zhang and\nSennrich, 2019) are the major two types of normalization layers used in Transformer architectures. LN'), Document(id='86a18daa-

In [11]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)



config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [12]:
compressed_docs = compression_retriever.invoke("What is Normalization?")

In [13]:
compressed_docs

[Document(id='e20025fd-8477-4945-9d26-cf4f1fce1ca1', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-03-14T01:17:11+00:00', 'author': '', 'keywords': '', 'moddate': '2025-03-14T01:17:11+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/transformer_without_normalization.pdf', 'total_pages': 19, 'page': 1, 'page_label': '2'}, page_content='normalization (Ulyanov et al., 2016), were initially proposed for specialized tasks such as object detection and\nimage stylization. They share the same overall formulation but differ in the axes and ranges over which the\nstatistics are computed.\nLayer normalization (LN) (Ba et al., 2016) and root mean square normalization (RMSNorm) (Zhang and\nSennrich, 2019) are the major two types of normalization layers used in Transformer architectures. LN'),
 Document(id='b48c57fe

In [14]:
# from langchain.document_loaders import TextLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
# from langchain.vectorstores import FAISS
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import CohereRerank
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
# from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [16]:
from langchain_groq import ChatGroq

In [17]:
llm = ChatGroq(temperature=0, model_name="gemma2-9b-it")

In [18]:
template = """
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()


In [ ]:
chain = (
    {"context": compression_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [21]:
# Define Prompt
template = """
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Say "I don't know" if the answer is not found in CONTEXT.

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

# Chain with explicit context retrieval
chain = (
    {
        "context": lambda x: "\n".join([doc.page_content for doc in compression_retriever.invoke(x)]),
        "query": RunnablePassthrough(),
    }
    | prompt
    | llm
    | output_parser
)

# Execute Query
query = "Which layer is replaced in transformer based on this paper?"
response = chain.invoke(query)
print(response)

Layer Norm (LN) layers 



In [22]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq

def load_and_split_pdf(pdf_path, chunk_size=500, chunk_overlap=100):
    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_documents(pages)

def create_retriever(texts, model_name="sentence-transformers/msmarco-distilbert-dot-v5", k=20):
    embeddings_model = HuggingFaceEmbeddings(model_name=model_name)
    return FAISS.from_documents(texts, embeddings_model).as_retriever(search_kwargs={"k": k})

def create_compression_retriever(retriever, model_name="BAAI/bge-reranker-base", top_n=3):
    model = HuggingFaceCrossEncoder(model_name=model_name)
    compressor = CrossEncoderReranker(model=model, top_n=top_n)
    return ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

def create_llm(model_name="gemma2-9b-it", temperature=0):
    return ChatGroq(temperature=temperature, model_name=model_name)

def create_prompt():
    template = """
    <|system|>>
    You are an AI Assistant that follows instructions extremely well.
    Please be truthful and give direct answers. Say "I don't know" if the answer is not found in CONTEXT.

    CONTEXT: {context}
    </s>
    <|user|>
    {query}
    </s>
    <|assistant|>
    """
    return ChatPromptTemplate.from_template(template)

def create_chain(retriever, llm):
    prompt = create_prompt()
    output_parser = StrOutputParser()
    return (
        {
            "context": lambda x: "\n".join([doc.page_content for doc in retriever.invoke(x)]),
            "query": RunnablePassthrough(),
        }
        | prompt
        | llm
        | output_parser
    )

def main(pdf_path, query):
    texts = load_and_split_pdf(pdf_path)
    retriever = create_retriever(texts)
    compression_retriever = create_compression_retriever(retriever)
    llm = create_llm()
    chain = create_chain(compression_retriever, llm)
    return chain.invoke(query)




In [23]:
if __name__ == "__main__":
    pdf_path = "/content/transformer_without_normalization.pdf"
    query = "What is Normalization?"
    response = main(pdf_path, query)
    print(response)

Normalization layers are used in deep learning to stabilize and accelerate training. 

They share a common formulation that involves normalizing the input data along specific axes and ranges. 

